In [1]:
include("HighDimMixedModels.jl")

Main.HighDimMM

In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [3]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
f = @formula(y ~  a + b + c + d)
contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)

Dict{Symbol, UnionAll} with 3 entries:
  :a => ContinuousTerm
  :b => ContinuousTerm
  :c => CategoricalTerm

In [ ]:
df

In [ ]:
## https://stackoverflow.com/questions/28756206/julia-converting-column-type-from-integer-to-float64-in-a-dataframe
df[:,:a] = convert(Vector{Float64},df[:,2]) # assign vector values and convert them into Int64
df[!,:a] = convert(Vector{Float64},df[:,2]) # change the vector reference to the vector with float64

In [ ]:
for i in 1:size(df,2)
    if(eltype(df[:,i]) == Int64)
        println("y")
        df[!,i] = convert(Vector{Float64},df[:,i])
    end
end

In [ ]:
df

In [ ]:
f

**try for f2**

In [ ]:
# try for f2
f2 = @formula(y ~ 1 + a + b + (1|c))

In [ ]:
sch = MixedModels.schema(f2,df,contrasts)

In [ ]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)

In [ ]:
form = MixedModels.apply_schema(f2, sch, HighDimMM.highDimMixedModel)

In [ ]:
form = apply_schema(f2, sch)

In [ ]:
?apply_schema

**back here**

In [38]:
sch = MixedModels.schema(f,df,contrasts)

StatsModels.Schema with 5 entries:
  c => c
  b => b
  d => d
  a => a
  y => y


In [41]:
form.rhs.terms[[2,3]]

b(continuous)
c(DummyCoding:3→2)

In [40]:
form = apply_schema(f, sch)

FormulaTerm
Response:
  y(continuous)
Predictors:
  a(continuous)
  b(continuous)
  c(DummyCoding:3→2)
  d(continuous)

In [55]:
typeof(form)

FormulaTerm{ContinuousTerm{Float64}, MatrixTerm{Tuple{ContinuousTerm{Float64}, ContinuousTerm{Float64}, CategoricalTerm{DummyCoding, String, 2}, ContinuousTerm{Float64}}}}

In [56]:
names(df)

5-element Vector{String}:
 "y"
 "a"
 "b"
 "c"
 "d"

In [93]:
findall(x -> x == "a", names(df))

1-element Vector{Int64}:
 2

In [111]:
if typeof(["a","b"]) == Array{String, 1}
    println("123")
else
    println("456")
end

123


In [109]:
typeof(["a","b"])

Vector{String} (alias for Array{String, 1})

In [ ]:
form.rhs.terms[[1,2]]

In [ ]:
modelcols(form.rhs.terms[[2,4]],df)

In [ ]:
modelcols(form.rhs.terms[1],df)

In [ ]:
Vector<:AbstractVector

In [ ]:
AbstractVecOrMat

In [ ]:
Vector(1:length(form.rhs.terms))

In [ ]:
[1,2]

In [ ]:
df[:,["b","c"]]

In [ ]:
y,pred = modelcols(form, df);

In [ ]:
y

In [ ]:
pred

In [ ]:
pred = modelmatrix(form, df);

In [ ]:
pred

In [ ]:
typeof(pred)

### Number of variables indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

In [ ]:
# Three matrix
HMM.M.M; HMM.X.X; HMM.Z.Z

### indices indicates M,X,Z

In [12]:
include("HighDimMixedModels.jl")

Main.HighDimMM

In [13]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([0.236781883208121 0.7522231521093703; 0.9437409715735081 0.31481548090434086; … ; 0.617492416565387 0.45458919106592477; 0.48153065407402607 0.38093255230858825]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

In [ ]:
Matrix{Int64}<:AbstractVecOrMat{AbstractFloat}

In [ ]:
Matrix{Int64}<:AbstractVecOrMat{<:Number}

In [ ]:
[1.0 2.3; 2.0 4.9]::AbstractVecOrMat{Float64}

In [ ]:
Matrix<:AbstractVecOrMat

In [14]:
HMM.M.M

9×1 Matrix{Float64}:
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
 9.0

In [15]:
HMM.X.X

9×2 Matrix{Float64}:
 0.236782  0.752223
 0.943741  0.314815
 0.445671  0.858522
 0.763679  0.698713
 0.145071  0.757746
 0.021124  0.419294
 0.152545  0.412607
 0.617492  0.454589
 0.481531  0.380933

In [ ]:
HMM.M.M * HMM.X.X

In [17]:
# response
y = HMM.y

9-element Vector{Float64}:
 0.5851946422124186
 0.07733793456911231
 0.7166282400543453
 0.3203570514066232
 0.6530930076222579
 0.2366391513734556
 0.7096838914472361
 0.5577872440804086
 0.05079002172175784

## variable names indicates M,X,Z

In [127]:
include("HighDimMixedModels.jl")

Main.HighDimMM

In [128]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "a", ["b","d"],"c")

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([0.236781883208121 0.7522231521093703; 0.9437409715735081 0.31481548090434086; … ; 0.617492416565387 0.45458919106592477; 0.48153065407402607 0.38093255230858825]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

In [129]:
HMM.M.M

9×1 Matrix{Float64}:
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
 9.0

In [130]:
HMM.X.X

9×2 Matrix{Float64}:
 0.236782  0.752223
 0.943741  0.314815
 0.445671  0.858522
 0.763679  0.698713
 0.145071  0.757746
 0.021124  0.419294
 0.152545  0.412607
 0.617492  0.454589
 0.481531  0.380933

In [131]:
HMM.Z.Z

9×2 Matrix{Float64}:
 0.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  0.0
 1.0  0.0
 0.0  1.0

In [132]:
y = HMM.y

9-element Vector{Float64}:
 0.5851946422124186
 0.07733793456911231
 0.7166282400543453
 0.3203570514066232
 0.6530930076222579
 0.2366391513734556
 0.7096838914472361
 0.5577872440804086
 0.05079002172175784

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$

To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

In [18]:
Z = HMM.Z.Z
y = HMM.y

9-element Vector{Float64}:
 0.5851946422124186
 0.07733793456911231
 0.7166282400543453
 0.3203570514066232
 0.6530930076222579
 0.2366391513734556
 0.7096838914472361
 0.5577872440804086
 0.05079002172175784

In [19]:
# A is X in formula
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
C = transpose(u[:,1:4]) # simplify: assume fixed effect full rank
K = C*P

4×9 Matrix{Float64}:
  8.85863e-18  -0.437769    0.0614729  …   0.417562  -0.118552   0.0472971
 -0.794874     -0.0213923   0.417183      -0.031295  -0.124852  -0.198574
 -1.68935e-17  -0.321258    0.479844      -0.551886   0.280829   0.243072
  8.20796e-17  -0.252174   -0.184915      -0.382227   0.017293   0.448349

In [20]:
K*A

4×3 Matrix{Float64}:
  1.38778e-15   3.84952e-17   1.11022e-16
 -3.88578e-16  -1.82847e-17  -5.55112e-17
 -1.11022e-15  -1.49561e-17  -1.66533e-16
  2.44249e-15   1.10253e-16   1.66533e-16

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [21]:
using NLopt

In [22]:
function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y) 
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    return negLog
end

negLogLik (generic function with 1 method)

In [ ]:
Sigma = HMM.Z*transpose(HMM.Z) + diagm(ones(length(y)))

In [ ]:
negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y

In [23]:
negLogLik([1.0,1.0], [2.0,2.0])

-0.7804182974389497

In [31]:
function negLogLik(sigma::Vector{Float64})
    n = length(y) 
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    return negLog
end

negLogLik([1.0,1.0])

-0.7804182974389497

In [24]:
opt = Opt(:LN_COBYLA, 2)
opt.min_objective = negLogLik
opt.xtol_rel = 1e-5

1.0e-5

In [ ]:
sigma = [1,1]
println("OPTBL: starting point $(sigma)")
(optf,optx,ret) = optimize(opt, sigma)
numevals = opt.numevals
println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $numevals iterations (returned $(ret))")

After getting estimate of sigma, we estimate beta
$$
\hat{\beta}(\theta) = (X^T\Sigma^{-1}X)^{-1}X^T\Sigma^{-1}y
$$

In [ ]:
sigma = [1, 1]
n = 9

In [ ]:
Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))

In [ ]:
# A is X in formula
inv(transpose(A)*inv(Sigma)*A)*transpose(A)*inv(Sigma)*y

In [ ]:
using NLopt

function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end
    return sqrt(x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end
    (a*x[1] + b)^3 - x[2]
end

opt = Opt(:LD_MMA, 2)
opt.lower_bounds = [-Inf, 0.]
opt.xtol_rel = 1e-4

opt.min_objective = myfunc
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,2,0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,-1,1), 1e-8)

minf,minx,ret = optimize(opt, [1.234, 5.678])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

In [ ]:
opt.

In [ ]:
function test()
    x = 1+2
    y =  2*x
    z = [1,2]
    return (x,y,z)
end

In [ ]:
test()

In [ ]:
x

In [ ]:
y

### test fit!()

In [36]:
include("HighDimMixedModels.jl")
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([0.236781883208121 0.7522231521093703; 0.9437409715735081 0.31481548090434086; … ; 0.617492416565387 0.45458919106592477; 0.48153065407402607 0.38093255230858825]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

In [37]:
HighDimMM.fit!(HMM, verbose = true, REML = true)

OPTBL: starting point [2, 2]
got 0.0 at [2.0, 2.0] after 1 iterations (returned FORCED_STOP)


([2.0, 2.0], [0.019822354833344942, -0.31637068786048494, 0.8854531460788502])